In [2]:
import numpy as np 
import cv2 as cv

In [3]:
def getLimits(color):
    
    c = np.uint8([[color]])
    hsvC = cv.cvtColor(c, cv.COLOR_BGR2HSV)
    
    lowerLimit = hsvC[0][0][0] - 10, 100, 100
    upperLimit = hsvC[0][0][0] + 10, 255, 255
    
    lowerLimit = np.array(lowerLimit, dtype=np.uint8)
    upperLimit = np.array(upperLimit, dtype=np.uint8)
    
    return lowerLimit, upperLimit
    

In [4]:
cap = cv.VideoCapture(0)

while True:
    ret, frame = cap.read()
    
    frame_blur = cv.GaussianBlur(frame, (7, 7), 2.0)
    
    hsv = cv.cvtColor(frame_blur, cv.COLOR_BGR2HSV)
    
    yellow = [0, 255, 255]
    pink   = [203, 192, 255]
    
    lowerP, upperP = getLimits(pink)
    lowerY, upperY = getLimits(yellow)
    
    maskPink = cv.inRange(hsv, lowerP, upperP)
    maskYellow = cv.inRange(hsv, lowerY, upperY)
    
    mask = cv.bitwise_or(maskPink, maskYellow)
    
    kernel = np.ones((5, 5), np.uint8)
    mask = cv.morphologyEx(mask, cv.MORPH_OPEN, kernel)
    
    contours, _ = cv.findContours(mask, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
    
    if contours:
        c = max(contours, key=cv.contourArea)
        x, y, w, h = cv.boundingRect(c)
        if(w > 20 and h > 20):
            frame = cv.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)

    cv.imshow('frame', frame)
    cv.imshow('mask', mask)
    
    if (cv.waitKey(1) & 0xFF == ord('q')):
        break
    
cap.release()
cv.destroyAllWindows()